PARA UTILIZAR GPU:

Presiona Entorno de ejecución o Runtime, luego en cambiar tipo. Ahí seleccionan GPU.

In [ ]:
import torch

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(DEVICE)

cuda:0
